In [ ]:
import numpy as np

import typing
from typing import Any, Tuple
import pandas as pd

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf

import string
from string import digits

import matplotlib.pyplot as plt
%matplotlib inline
import re
# from matplotlib.font_manager import FontProperties
# font_prop = FontProperties(fname='Mangal.ttf', size=18)

In [ ]:
import tensorflow_text as tf_text

In [ ]:
lines=pd.read_csv("D:\\Final_Year\\NLP Translator\\hindi_to_english_dataset.csv",encoding='utf-8')

In [ ]:
pd.isnull(lines).sum()

In [ ]:
lines=lines[~pd.isnull(lines['english'])]
lines=lines[~pd.isnull(lines['hindi'])]

In [ ]:
lines.drop_duplicates(inplace=True)

In [ ]:
lines.shape

In [ ]:
hindi=np.array(lines['hindi'][110564:])
english=np.array(lines['english'][110564:])

In [ ]:
def clean_english_data(sentence):
    exclude = set(string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    sentence = sentence.lower()
    sentence = ''.join(ch for ch in sentence if ch not in exclude)
    sentence = sentence.translate(remove_digits)
    sentence = sentence.strip()
    sentence = re.sub(" +", " ", sentence)
    return sentence

In [ ]:
def clean_hindi_data(sentence):
    exclude = set(string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    sentence = sentence.lower()
    sentence = ''.join(ch for ch in sentence if ch not in exclude)

    sent_temp = ''
    for c in sentence:
        if c == ' ':
            sent_temp += c
        elif ord(u'\u0900') <= ord(c) <= ord(u'\u097F'):
            sent_temp += c
    sentence = sent_temp
      
    sentence = re.sub('[a-z]', '', sentence)
    sentence = re.sub('[०१२३४५६७८९।]', '', sentence)
    sentence = sentence.translate(remove_digits)
    sentence = sentence.strip()
    sentence = re.sub(" +", " ", sentence)
    return sentence

In [ ]:
hindi

In [ ]:
english

In [ ]:
context_raw=np.array([clean_english_data(x) for x in english])
target_raw=np.array([clean_hindi_data(x) for x in hindi])

In [ ]:
print(target_raw.shape)
print(target_raw[-1])

In [ ]:
print(context_raw[-1])

Creating Tensorflow Dataset

In [ ]:

BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [ ]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

In [ ]:
example_text = tf.constant('सिंगापुर के फरेर पार्क में रासबिहारी बोस ने भारतीय स्वतंत्रता परिषद का नेतृत्व सुभाषबाबू को सौंप दिया।')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFD').numpy())

In [ ]:
def tf_lower_and_split_punct(text):
  # Split accented characters.
  
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  text = tf_text.normalize_utf8(text, 'NFD')
  return text

In [ ]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())